Import data to dataset and result format for sklearn

In [12]:
import pandas as pd
df = pd.read_csv('../data/train.csv')
dataset = []
result = []
for index, row in df.iterrows():
    data = []
    counter = 0
    for column in row:
        counter += 1
        if counter != 1:
            if counter == 2:
                result.append(column)
            else:
                
                data.append(column)
    dataset.append(data)

Normalization each column

In [9]:
max_value_dataset = []
min_value_dataset = []
first_iter = True
for row in dataset:
    counter = 0
    for value in row:
        if first_iter:
            max_value_dataset.append(value)
            min_value_dataset.append(value)
        else:
            if max_value_dataset[counter]<value:
                max_value_dataset[counter] = value
            if min_value_dataset[counter]>value:
                min_value_dataset[counter] = value
        counter += 1
    first_iter = False

In [10]:
print dataset[0][1]
for row in dataset:
    counter = 0
    for value in row:
        row[counter] = float(value-min_value_dataset[counter])/float(max_value_dataset[counter]-min_value_dataset[counter])*9+1  
        counter += 1
print dataset[0][1]

19.89
4.81668946648


Export data to CSV

In [11]:
headers = ['B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z','AA','AB','AC']
counter = 0
for value in result:
    dataset[counter] = [value] + dataset[counter]
    counter += 1
df = pd.DataFrame(dataset,columns=headers)
df.to_csv('../data/normalized_train.csv')

Normalization consider same type

In [13]:
#                    wh         celcius                     kelembapan          mmgh m/s   km  A/c  rnd1 rnd2
same_type_columns = [[0],[1,3,5,7,9,11,13,15,17,19],[2,4,6,8,10,12,14,16,18,21],[20],[22],[23],[24],[25],[26]]
max_value_dataset = [0,0,0,0,0,0,0,0,0]
min_value_dataset = [0,0,0,0,0,0,0,0,0]
first_iter = True
for row in dataset:
    counter = 0
    for value in row:
        if first_iter:
            counter_temp = 0
            for i in same_type_columns:
                if counter in i:
                    max_value_dataset[counter_temp] = value
                    min_value_dataset[counter_temp] = value 
                    break
                counter_temp += 1
        else:
            counter_temp = 0
            for i in same_type_columns:
                if counter in i:
                    if max_value_dataset[counter_temp]<value:
                        max_value_dataset[counter_temp] = value
                    if min_value_dataset[counter_temp]>value:
                        min_value_dataset[counter_temp] = value
                    break
                counter_temp += 1
        counter += 1
    first_iter = False

NameError: name 'composite' is not defined

In [53]:
print dataset[0][1]
for row in dataset:
    counter = 0
    for value in row:
        counter_temp = 0
        for i in composite:
            if counter in i:
                row[counter] = float(value-min_value_dataset[counter_temp])/float(max_value_dataset[counter_temp]-min_value_dataset[counter_temp])*9+1  
            counter_temp += 1
        counter += 1
print dataset[0][1]

19.89
8.12940637876


Export data to CSV Per Type

In [ ]:
headers = ['B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z','AA','AB','AC']
counter = 0
for value in result:
    dataset[counter] = [value] + dataset[counter]
    counter += 1
df = pd.DataFrame(dataset,columns=headers)
df.to_csv('../data/normalized_per_type_train.csv')

Feature Selection using sklearn score

In [23]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2 # all positive
from sklearn.feature_selection import f_classif # can negative
from sklearn.feature_selection import mutual_info_classif # can negative

selector = SelectKBest(f_classif, k='all').fit(dataset,result)
x_new = selector.transform(dataset) # not needed to get the score
scores = selector.scores_
print scores

SVM Implementation Regression

In [14]:
from sklearn import svm
clf = svm.SVR()
clf.fit(dataset, result) 
prediction = clf.predict(dataset)

Evaluation MSE

In [15]:
from sklearn.metrics import mean_squared_error
mean_squared_error(result, prediction)

12608.809438271872